## imports

In [1]:
import architectures as ar
import torch
from torch import nn

## model architecture tests

In [2]:
args = []
dilations = [2,4,8,8,4]
strides=[5,4,3,2,2]
kernels=[10,8,4,4,4]
encode_dim=32


In [3]:
vicreg = ar.VicRegWaves(args,dilations,strides,kernels,encode_dim,[32,64,128,64,32])

In [4]:
bsz = 128
a1 = [torch.normal(torch.sin(torch.linspace(0,2*torch.pi,441)),torch.tensor([1])) for _ in range(bsz)]
a1 = torch.stack(a1,axis=0)[:,None,:]
print(a1.shape)

torch.Size([128, 1, 441])


In [5]:
def ProjectorTest(sizes):

    layers = []
    for ii in range(len(sizes) - 2):

        layers.append(nn.Linear(sizes[ii],sizes[ii+1]))
        layers.append(nn.BatchNorm1d(sizes[ii+1]))
        layers.append(nn.ReLU(True))
    layers.append(nn.Linear(sizes[-2],sizes[-1],bias=False))
    return nn.Sequential(*layers),layers

In [6]:
senc = vicreg.encoder.strided_encoder
denc = vicreg.encoder.dilated_encoder
proj = vicreg.projector
#sizes = vicreg.sizes
projTest,layers = ProjectorTest([32,64,128,64,32])

In [11]:
d1 = denc(a1)
s1 = senc(a1)
e1 = vicreg.encoder(a1)
ps = [e1]
for ii,layer in enumerate(layers):
    try:
        ps.append(layer(ps[ii]))
    except:
        print(f"failed on layer {ii+1}: input shape {ps[ii].shape}")
        layer(ps[ii])

loss = vicreg(a1[:64,:],a1[:64,:])

In [12]:
print(d1.shape,s1.shape,e1.shape,ps[-1].shape)

torch.Size([128, 16, 1]) torch.Size([128, 16, 1]) torch.Size([128, 32]) torch.Size([128, 32])


## dataset tests

In [1]:
import os
import glob 
import data as dt
import numpy as np

path = '/home/miles/mooney_server/Miles/fromComputer/birds_jq/CAGbirds/org545/data'
days = glob.glob(os.path.join(path,'[0-9]*'))
audio = [glob.glob(os.path.join(d,'denoised','*.wav')) for d in days]
rois = [glob.glob(os.path.join(d,'denoised_segments_syll','*.txt')) for d in days]
audio = sum(audio,[])
rois = sum(rois,[])

In [2]:
ds1 = dt.waveformSet(audio,rois,maxSegs=50,seed=100)


Pre-processing audio: 0it [00:00, ?it/s]


/home/miles/mooney_server/Miles/fromComputer/birds_jq/CAGbirds/org545/data/20220301/denoised_segments_syll/recording_20220301_131747nrd.txt
max number of segments reached!
Getting positive pair indices


being inefficient: 516it [00:19, 26.80it/s]


In [8]:
r = rois[0]
x,y = np.loadtxt(r,delimiter=' ',unpack=True)
print(x.shape,y.shape)

(69,) (69,)
